### Loading the Data 

In [1]:
import numpy as np
from posttestFunctions import *


import os 
path =r"C:\Users\barak\Documents\GitHub\startegicMW_lite\Experiment\data"
filename = "MDEXP"

### Load Particpant data
[rawEye, events, gazeCoords] = loadParticipant(path, name=filename) # Gets my RawEye Data as well as events and gaze Coordination
RawDF = pd.DataFrame(rawEye,columns=["TimePoint",'LeftX','LeftY','LeftPupil','RightX','RightY','RightPupil']);

### Get Paricular Events

import pandas as pd
pd.set_option("display.max_rows",10)
fixationDF,saccadesDF,blinkDF = getEvent(events)


def changeTiming(RawDF,fixationDF,saccadesDF,blinkDF):
    # Get the time of the event

    fixationDF['Beg'] = fixationDF['Beg'] -RawDF['TimePoint'].iloc[0]
    fixationDF['End'] = fixationDF['End'] -RawDF['TimePoint'].iloc[0]

    saccadesDF['Beg'] = saccadesDF['Beg'] -RawDF['TimePoint'].iloc[0]
    saccadesDF['End'] = saccadesDF['End'] -RawDF['TimePoint'].iloc[0]

    blinkDF['Beg'] = blinkDF['Beg'] -RawDF['TimePoint'].iloc[0]
    blinkDF['End'] = blinkDF['End'] -RawDF['TimePoint'].iloc[0]

    RawDF['TimePoint'] = RawDF['TimePoint'] - RawDF['TimePoint'].iloc[0]



    return RawDF,fixationDF,saccadesDF,blinkDF

RawDF,fixationDF,saccadesDF,blinkDF = changeTiming(RawDF,fixationDF,saccadesDF,blinkDF)

In [ ]:
fixationDF

### General Checks 

In [ ]:
###  Plot Raw Data
from posttestFunctions import eyeRawKDE,plotRawGaze


eyeRawKDE(RawDF)
plt.savefig(os.path.join(path,filename,filename+"_Distribution.png"))
plotRawGaze(RawDF,gazeCoords)
plt.savefig(os.path.join(path,filename,filename+"_RawGaze.png"))


In [ ]:
log_file = open(os.path.join(path,filename,filename+"_analysisLog.txt"),'w+')
log_file.write(" "*15+"="*58 + "\n")
log_file.write(" "*15+"="*5+ " "*15+" ANALYSIS LOG FILE"+ " "*15+"="*5+ " \n")
log_file.write(" "*15+"="*58 + "\n")


In [ ]:

log_file.write("\n"+"="*30 + "\n")
log_file.write(" General Data Check")

log_file.write("\n"+"="*30 + "\n")
log_file.write("\n"+"-"*100 + "\n")

ScreenData = RawDF[((RawDF['LeftX'].values > 0) & (RawDF['LeftX'].values < gazeCoords[2])) & 
                   ((RawDF['LeftY'].values > 0) & (RawDF['LeftY'].values < gazeCoords[3]))]
RawDF.to_csv(os.path.join(path, filename, filename + "_RawGaze.csv"))  # Save to CSV for further analysis

# 0. Descriptive Stats
print("\n(0) Descriptive Statistics of Raw Data\n")
log_file.write("\n(0) Descriptive Statistics of Raw Data\n")
print("For Raw Data")
log_file.write("For Raw Data\n")
print(RawDF.describe())
log_file.write(RawDF.describe().to_string() + "\n")
print("\nFor Screen Confined Data")
log_file.write("\nFor Screen Confined Data\n")
print(ScreenData.describe())
log_file.write(ScreenData.describe().to_string() + "\n")

# 3. Check Percentage of Missing Data, Out of bounds data
print("\n(1) Check Missing Data Gaze Center\n")
log_file.write("\n(1) Check Missing Data Gaze Center\n")
print(f"Percentage of Out of Screen Events: {(len(RawDF) - len(ScreenData)) / len(RawDF):.4f}")
log_file.write(f"Percentage of Out of Screen Events: {(len(RawDF) - len(ScreenData)) / len(RawDF):.4f}\n")
print(f"Percentage of Missing Eye Events:   {len(RawDF[(RawDF['RightPupil'] == 0) | (RawDF['LeftPupil'] == 0)]) / len(RawDF):.4f}")
log_file.write(f"Percentage of Missing Eye Events:   {len(RawDF[(RawDF['RightPupil'] == 0) | (RawDF['LeftPupil'] == 0)]) / len(RawDF):.4f}\n")

# 2. Check whether Majority of Gaze Positions Lay in the center
print("\n(2) Check mean Gaze Center\n")
log_file.write("\n(2) Check mean Gaze Center\n")

print("                              X          Y")
log_file.write("                              X           Y\n")
print(f"Middle Screen Coordinates: {(gazeCoords[2] / 2):6.1f}      {gazeCoords[3] / 2:6.1f}")
log_file.write(f"Middle Screen Coordinates: {(gazeCoords[2] / 2):6.1f}      {gazeCoords[3] / 2:6.1f}\n")
print(f"Mean Eye Coordinates:      {np.mean(ScreenData['LeftX']):5.1f}       {np.mean(ScreenData['LeftY']):5.1f}")
log_file.write(f"Mean Eye Coordinates:      {np.mean(ScreenData['LeftX']):5.1f}       {np.mean(ScreenData['LeftY']):5.1f}\n")

# 1. Check if there are Too Many Missing data from One Eye
print("\n(3) Check Missing Pupil Data as a proxy for missing!\n")
log_file.write("\n(3) Check Missing Pupil Data as a proxy for missing!\n")
zeroPupilLeft = RawDF[RawDF['LeftPupil'] == 0]
zeroPupilRight = RawDF[RawDF['RightPupil'] == 0]
zeroPupilBoth = RawDF[(RawDF['RightPupil'] == 0) & (RawDF['LeftPupil'] == 0)]
print(f"Number of NoGaze Data: \n(Left: {len(zeroPupilLeft)}, Right: {len(zeroPupilRight)}), Both: {len(zeroPupilBoth)}")
log_file.write(f"Number of NoGaze Data: \n(Left: {len(zeroPupilLeft)}, Right: {len(zeroPupilRight)}), Both: {len(zeroPupilBoth)}\n")

log_file.write("\n"+"-"*100 + "\n")


# Print a message indicating that the log file has been created


### Fixation Checks

In [1]:
from posttestFunctions import plotFixData
import sys
fixationDF.to_csv(os.path.join(path,filename,filename+"_RawFix.csv"))

fixPosL = fixationDF[fixationDF['Eye'] =='L']
fixPosR = fixationDF[fixationDF['Eye'] =='R']

#Before Matching the data
titl = "Raw Fixations"
plotFixData(fixPosL,fixPosR,gazeCoords,titl="Raw Fixations")
plt.savefig(os.path.join(path,filename,filename+"_RawFix.png"))

fixPosL, fixPosR, matched_fixation_df = matchLeftRight(fixationDF)
# Plot After Matching The Data
plotFixData(fixPosL,fixPosR,gazeCoords,titl="Matched Fixations")


KeyboardInterrupt: 

In [ ]:
# Preamble for Fixation Checks
log_file.write("\n" + "="*30 + "\n")
log_file.write(" Fixation Data Check")
log_file.write("\n" + "="*30 + "\n")
log_file.write("\n" + "-"*100 + "\n")

### FIXATION CHECKS:
from posttestFunctions import cleanStd

# Descriptive Stats and checks:
print()
print("(0) Descriptive Statistics of Fixation Events")
log_file.write("\n(0) Descriptive Statistics of Fixation Events\n")
print("\n                                      LeftEye")
log_file.write("\n                                      LeftEye\n")
print(fixPosL[fixPosL.columns[1:]].describe())
log_file.write(fixPosL[fixPosL.columns[1:]].describe().to_string() + "\n")
print("\n                                      RightEye")
log_file.write("\n                                      RightEye\n")
print(fixPosR[fixPosL.columns[1:]].describe())
log_file.write(fixPosR[fixPosL.columns[1:]].describe().to_string() + "\n")

# 0. Check whether the Number of Fixation Match:
print()
print("(1) Check Left Right Number Match")
log_file.write("\n(1) Check Left Right Number Match\n")
if len(fixPosL) != len(fixPosR):
    print(f"    !WARNING! Fixations do not Match! \n    (Left: {len(fixPosL)} Right {len(fixPosR)})")
    log_file.write(f"    !WARNING! Fixations do not Match! \n    (Left: {len(fixPosL)} Right {len(fixPosR)})\n")
    print(f"    Something Is Off In Calibration or Events Computation")
    log_file.write(f"    Something Is Off In Calibration or Events Computation\n")
else:
    print(f"    Fixation Number is Matched Between the Eyes \n    (Left: {len(fixPosL)} Right {len(fixPosR)})")
    log_file.write(f"    Fixation Number is Matched Between the Eyes \n    (Left: {len(fixPosL)} Right {len(fixPosR)})\n")

# 1. Check Delay Between Fixations:
print()
print("(2) Check Right-Left Temporal Delay")
log_file.write("\n(2) Check Right-Left Temporal Delay\n")
maxDelayBeg = max(abs(fixPosL['Beg'].values - fixPosR['Beg'].values))
maxDelayEnd = max(abs(fixPosL['End'].values - fixPosR['End'].values))

if maxDelayBeg < 50:
    print(f"    Max Beginning Right-Left Delay Is below 50 ms:  {maxDelayBeg}")
    log_file.write(f"    Max Beginning Right-Left Delay Is below 50 ms:  {maxDelayBeg}\n")
else:
    print(f"    WARNING! Max Right-Left Delay Is ABOVE 50 ms:  {maxDelayBeg}")
    log_file.write(f"    WARNING! Max Right-Left Delay Is ABOVE 50 ms:  {maxDelayBeg}\n")

if maxDelayEnd < 50:
    print(f"    Max End Right-Left Delay Is below 50 ms:        {maxDelayEnd}")
    log_file.write(f"    Max End Right-Left Delay Is below 50 ms:        {maxDelayEnd}\n")
else:
    print(f"    WARNING! Max Right-Left Delay Is ABOVE 50 ms:  {maxDelayEnd}")
    log_file.write(f"    WARNING! Max Right-Left Delay Is ABOVE 50 ms:  {maxDelayEnd}\n")

# 2. Check Left-Right Position Delay
print()
print("(3) Check Right-Left Spatial Delay")
log_file.write("\n(3) Check Right-Left Spatial Delay\n")
maxDelayX = abs(fixPosL['X'].values - fixPosR['X'].values)
maxDelayY = abs(fixPosL['Y'].values - fixPosR['Y'].values)
delayedFix = []
for i in range(len(maxDelayX)):
    if maxDelayX[i] > 50:
        print(f"    WARNING! Width Right-Left Delay Is ABOVE 50 pix:    (index: {i}, {maxDelayX[i]:0.1f})")
        log_file.write(f"    WARNING! Width Right-Left Delay Is ABOVE 50 pix:    (index: {i}, {maxDelayX[i]:0.1f})\n")
        delayedFix.append(i)
    if maxDelayY[i] > 25:
        print(f"    WARNING! Height Right-Left Delay Is ABOVE 25 pix:   (index: {i}, {maxDelayY[i]:0.1f})")
        log_file.write(f"    WARNING! Height Right-Left Delay Is ABOVE 25 pix:   (index: {i}, {maxDelayY[i]:0.1f})\n")
        delayedFix.append(i)

print(f"   Number of Left-Right Position Disparity Issues ({len(set(delayedFix))})")
log_file.write(f"   Number of Left-Right Position Disparity Issues ({len(set(delayedFix))})\n")
print(f"   Artefactual Trials Consist of almost {len(set(delayedFix)) / len(fixPosL) * 100}% of All!!!")
log_file.write(f"   Artefactual Trials Consist of almost {len(set(delayedFix)) / len(fixPosL) * 100}% of All!!!\n")

# 3. Check Left-Right Position Delay
print()
print("(4) Check Fixation Duration")
log_file.write("\n(4) Check Fixation Duration\n")
durationL = fixPosL['End'].values - fixPosL['Beg'].values
durationR = fixPosR['End'].values - fixPosR['Beg'].values

artefacts = []
for i in range(len(fixPosL)):
    if durationL[i] < 50:
        print(f"    Warning! Duration of Left Saccade is Wrong ({durationL[i]}) \n        (index: {i}, beg: {fixPosL.iloc[i]['Beg']}, end: {fixPosL.iloc[i]['End']:0.1f})")
        log_file.write(f"    Warning! Duration of Left Saccade is Wrong ({durationL[i]}) \n        (index: {i}, beg: {fixPosL.iloc[i]['Beg']}, end: {fixPosL.iloc[i]['End']:0.1f})\n")
        artefacts.append(i)
    if durationR[i] < 50:
        print(f"    Warning! Duration of Right Saccade is Wrong ({durationR[i]})\n        (index: {i}, beg: {fixPosR.iloc[i]['Beg']}, end: {fixPosR.iloc[i]['End']:0.1f})")
        log_file.write(f"    Warning! Duration of Right Saccade is Wrong ({durationR[i]})\n        (index: {i}, beg: {fixPosR.iloc[i]['Beg']}, end: {fixPosR.iloc[i]['End']:0.1f})\n")
        artefacts.append(i)
    if durationL[i] - durationR[i] > 20:
        print(f"    Warning! Mismatch Between Left and Right Fix Duration \n        (index: {i}, Left: {durationL[i]}, Right: {durationR[i]})")
        log_file.write(f"    Warning! Mismatch Between Left and Right Fix Duration \n        (index: {i}, Left: {durationL[i]}, Right: {durationR[i]})\n")
    if durationL[i] - durationR[i] > 40:
        artefacts.append(i)

if len(artefacts) == 0:
    print("\n    No Artefacts in Fixation Duration")
    log_file.write("\n    No Artefacts in Fixation Duration\n")
else:
    print(f"\n    Duration Artefact Rate {len(artefacts) / len(fixPosL) * 100}%")
    log_file.write(f"\n    Duration Artefact Rate {len(artefacts) / len(fixPosL) * 100}%\n")

# 4. Check for X and Y to Screen Outliers:: (Via Plot Actually:)
### - Checking also by thresholding by 3std - there should have been any infringements!
print("\n(5) Check For Outliers in our Data")
log_file.write("\n(5) Check For Outliers in our Data\n")
print("\n     Outlier Fixation: \n")
log_file.write("\n     Outlier Fixation: \n\n")

print(cleanStd(fixationDF, ['X', 'Y'], 3))
log_file.write(cleanStd(fixationDF, ['X', 'Y'], 3).to_string() + "\n")

print("\n     Events Outside of the Screen: \n")
log_file.write("\n     Events Outside of the Screen: \n\n")
print(fixationDF[(list(fixationDF['X'].values < 0) or list(fixationDF['X'].values > gazeCoords[2])) or (list(fixationDF['Y'].values < 0) or list(fixationDF['Y'].values > gazeCoords[3]))])
log_file.write(fixationDF[(list(fixationDF['X'].values < 0) or list(fixationDF['X'].values > gazeCoords[2])) or (list(fixationDF['Y'].values < 0) or list(fixationDF['Y'].values > gazeCoords[3]))].to_string() + "\n")

log_file.write("\n" + "-"*100 + "\n")
# Print a message indicating that the fixation checks have been written to the log file
print("Fixation checks have been written to the log file.")

### Pupil Checks and Preprocessing

In [ ]:
# Preamble for Pupil Diameter Checks
log_file.write("\n" + "="*30 + "\n")
log_file.write(" Pupil Diameter Checks")
log_file.write("\n" + "="*30 + "\n")
log_file.write("\n" + "-"*100 + "\n")

#### --- Pupil Diameter Checks ---- #####
from scipy.stats import wilcoxon
from posttestFunctions import removeBlinks, cleanStd, cleanIQR

print("\n (0) Pupil Descriptive Stats \n")
log_file.write("\n (0) Pupil Descriptive Stats \n")
print(ScreenData[['LeftPupil', 'RightPupil']].describe())
log_file.write(ScreenData[['LeftPupil', 'RightPupil']].describe().to_string() + "\n")

# (1) Mismatch between Pupil Diameters Left to Right 
# Get raw when all data are greater than 0 (no blinks) 
RawDFPupilTrue = ScreenData[(ScreenData['LeftPupil'] > 0) & (ScreenData['RightPupil'] > 0)]
pupilDifference = (RawDFPupilTrue['LeftPupil'] - RawDFPupilTrue['RightPupil']) / RawDFPupilTrue['LeftPupil']

print("\n (1) Pupil Difference Test \n")
log_file.write("\n (1) Pupil Difference Test \n")
print("    Pupil Difference Test (May Be unreliable) : \n    " + str(wilcoxon(RawDFPupilTrue['LeftPupil'], RawDFPupilTrue['RightPupil'])))
log_file.write("    Pupil Difference Test (May Be unreliable) : \n    " + str(wilcoxon(RawDFPupilTrue['LeftPupil'], RawDFPupilTrue['RightPupil'])) + "\n")
print(f"    Percentage Difference: {np.mean(pupilDifference)}")
log_file.write(f"    Percentage Difference: {np.mean(pupilDifference)}\n")

# (2) Pupil Diameter Through Time 
import pandas as pd
print("\n (2) Pupil Diameter Through Time \n")
log_file.write("\n (2) Pupil Diameter Through Time \n")

# Before Removing the Blinks
plotPupilTimecourse(ScreenData, "Raw Pupil Data with Blinks", blinkDF, saccadesDF, fixationDF, chooseViz='111')
plt.savefig(os.path.join(path,filename,filename+"_RawPupil.png"))

gaze_df_filtered = removeBlinks(blinkDF, ScreenData)
print(gaze_df_filtered)
#log_file.write(gaze_df_filtered.to_string() + "\n")


# After Removing the Blinks
plotPupilTimecourse(gaze_df_filtered, "Filtered Pupil Data", blinkDF, saccadesDF, fixationDF, chooseViz='001')
plt.savefig(os.path.join(path,filename,filename+"__BlinklessPupil.png"))

# (3) List Pupil Artefacts (via IQR and 3std)
print("\n (3) List IQR Events: ")
log_file.write("\n (3) List IQR Events: \n")
print(f"\n    Mean Pupil Diameter: Left {np.mean(ScreenData['LeftPupil'].values):7.2f}; Right {np.mean(ScreenData['RightPupil'].values):7.2f}")
log_file.write(f"\n    Mean Pupil Diameter: Left {np.mean(ScreenData['LeftPupil'].values):7.2f}; Right {np.mean(ScreenData['RightPupil'].values):7.2f}\n")
print(f"     Std Pupil Diameter: Left {np.std(ScreenData['LeftPupil'].values):7.2f}; Right {np.std(ScreenData['RightPupil'].values):7.2f}\n")
log_file.write(f"     Std Pupil Diameter: Left {np.std(ScreenData['LeftPupil'].values):7.2f}; Right {np.std(ScreenData['RightPupil'].values):7.2f}\n")

print("\n        Artefactual Pupil Data")
log_file.write("\n        Artefactual Pupil Data\n")
print(cleanStd(ScreenData, ['RightPupil', 'LeftPupil'], 3)[['TimePoint', 'RightPupil', 'LeftPupil']])
log_file.write(cleanStd(ScreenData, ['RightPupil', 'LeftPupil'], 3)[['TimePoint', 'RightPupil', 'LeftPupil']].to_string() + "\n")

IQRartifs, ScreenDataCleaned = cleanIQR(ScreenData, 'LeftPupil', threshold=1.5)

log_file.write("\n" + "-"*100 + "\n")

# Print a message indicating that the pupil diameter checks have been written to the log file
print("Pupil diameter checks have been written to the log file.")

In [ ]:
from posttestFunctions import plotSaccadeData,checkSaccadeMismatch

# Preamble for Saccade Checks
log_file.write("\n" + "="*30 + "\n")
log_file.write(" Saccade Data Check")
log_file.write("\n" + "="*30 + "\n")
log_file.write("\n" + "-"*100 + "\n")

#### --- Visualize Saccades ---- ###

print(saccadesDF.describe())
log_file.write(saccadesDF.describe().to_string() + "\n")


# Match left and right Saccades
saccPosL, saccPosR, matched_saccades_df = matchLeftRight(saccadesDF)
plotSaccadeData(saccPosL, saccPosR, gazeCoords);
plt.savefig(os.path.join(path,filename,filename+"_Saccades.png"))

# (1) Check StartX and StartY as well as End Between Left and Right Eye
print("\n(1) Check The Left Right Saccade Differences")
log_file.write("\n(1) Check The Left Right Saccade Differences\n")

artifSaccadesPos = []
for i in range(len(saccPosL)):
    currentLeft = saccPosL.iloc[i]
    currentRight = saccPosR.iloc[i]

    # StartX delay
    stXdelay = abs(currentLeft['StartX'] - currentRight['StartX'])
    stYdelay = abs(currentLeft['StartY'] - currentRight['StartY'])
    endXdelay = abs(currentLeft['EndX'] - currentRight['EndX'])
    endYdelay = abs(currentLeft['EndY'] - currentRight['EndY'])

    if stXdelay > 50:
        print(f"    Delay in Beg X for {i}: ( {currentLeft['StartX']} -  {currentRight['StartX']} = {stXdelay:5.2f})")
        log_file.write(f"    Delay in Beg X for {i}: ( {currentLeft['StartX']} -  {currentRight['StartX']} = {stXdelay:5.2f})\n")
        artifSaccadesPos.append(i)

    if stYdelay > 25:
        print(f"    Delay in Beg Y for {i}: ( {currentLeft['StartY']} -  {currentRight['StartY']} = {stYdelay:5.2f})")
        log_file.write(f"    Delay in Beg Y for {i}: ( {currentLeft['StartY']} -  {currentRight['StartY']} = {stYdelay:5.2f})\n")
        artifSaccadesPos.append(i)

    if endXdelay > 50:
        print(f"    Delay in End X for {i}: ( {currentLeft['EndX']} -  {currentRight['EndX']} = {endXdelay:5.2f})")
        log_file.write(f"    Delay in End X for {i}: ( {currentLeft['EndX']} -  {currentRight['EndX']} = {endXdelay:5.2f})\n")
        artifSaccadesPos.append(i)

    if endYdelay > 25:
        print(f"    Delay in End Y for {i}: ( {currentLeft['EndY']} -  {currentRight['EndY']} = {endYdelay:5.2f})")
        log_file.write(f"    Delay in End Y for {i}: ( {currentLeft['EndY']} -  {currentRight['EndY']} = {endYdelay:5.2f})\n")
        artifSaccadesPos.append(i)

print(f"\n    Artifact to saccade ratio: {len(artifSaccadesPos) / len(saccPosL):5.2f} (N: {len(saccPosL)} {len(artifSaccadesPos)})")
log_file.write(f"\n    Artifact to saccade ratio: {len(artifSaccadesPos) / len(saccPosL):5.2f} (N: {len(saccPosL)} {len(artifSaccadesPos)})\n")

# (2) Check Saccades Duration: (Best thing to filter Out!!!)
print("\n(2) Check The Duration of the Saccade")
log_file.write("\n(2) Check The Duration of the Saccade\n")

artifactualSaccadesDur = []
for i in range(len(saccPosL)):
    currentLeft = saccPosL.iloc[i]['Duration']

    if currentLeft > 120:
        print(f"    Saccade in {i} is too long ({currentLeft} ms ({saccPosL.iloc[i]['Beg']}))")
        log_file.write(f"    Saccade in {i} is too long ({currentLeft} ms ({saccPosL.iloc[i]['Beg']}))\n")
        artifactualSaccadesDur.append(i)

    if currentLeft < 5:
        print(f"    Saccade in {i} is too short ({currentLeft} ms ({saccPosL.iloc[i]['Beg']}))")
        log_file.write(f"    Saccade in {i} is too short ({currentLeft} ms ({saccPosL.iloc[i]['Beg']}))\n")
        artifactualSaccadesDur.append(i)

# Visualize Faulty Saccades on a Time Course to See their Impact on other Events!
artifactualSaccades = saccPosL.iloc[artifactualSaccadesDur]
log_file.write("\n")
print()
print(artifactualSaccades)
log_file.write(artifactualSaccades.to_string() + "\n")
plotPupilTimecourse(ScreenData, "Raw Pupil Data with Too High Saccades", blinkDF, artifactualSaccades, fixationDF, chooseViz='010')
plt.savefig(os.path.join(path, filename, filename + "_HighSaccades.png"))
# (3) Check the mismatch between the End and the beginning of the next

print("     Too long Saccade Data:")
log_file.write("     Too long Saccade Data:")


    
print("\n     Duration Descriptive: ")
log_file.write("\n     Duration Descriptive: ")

log_file.write("\n")
print()
saccPosL2 = checkSaccadeMismatch(saccPosL)
print(saccPosL2[['diffX', 'diffY']].describe())
log_file.write(saccPosL2[['diffX', 'diffY']].describe().to_string() + "\n")
nFaultySac = len(saccPosL2[(saccPosL2['diffX'] > 10) | (saccPosL2['diffY'] > 10)])

print(f"\n    Number of Mismatched Saccades in Relation to All: {nFaultySac/len(saccPosL2):5.2f}")
log_file.write(f"\n    Number of Mismatched Saccades in Relation to All: {nFaultySac/len(saccPosL2):5.2f}\n")

log_file.write("\n" + "-"*100 + "\n")

# Print a message indicating that the saccade checks have been written to the log file
print("Saccade checks have been written to the log file.")

In [ ]:
# Preamble for Blink Checks
log_file.write("\n" + "="*30 + "\n")
log_file.write(" Blink Data Check")
log_file.write("\n" + "="*30 + "\n")
log_file.write("\n" + "-"*100 + "\n")

#### ---- Check Blinks ----- #####
print("\n(1) Descriptive Stats of Blinks \n")

print(blinkDF.describe())
log_file.write(blinkDF.describe().to_string() + "\n")

print(f"Blink Events for Eyes: Left: {len(blinkDF[blinkDF['Eye'] == 'L'])} Right: {len(blinkDF[blinkDF['Eye'] == 'R'])}")
log_file.write(f"Blink Events for Eyes: Left: {len(blinkDF[blinkDF['Eye'] == 'L'])} Right: {len(blinkDF[blinkDF['Eye'] == 'R'])}\n")

# Match left and right Blinks
blinkPosL, blinkPosR, matched_blink_df = matchLeftRight(blinkDF)

# (1) Check Whether Blinks Left and Right are Truly The Same (Pointless)
# blinkDF[blinkDF['Eye'] == 'L']['Beg'].values - blinkDF[blinkDF['Eye'] == 'R']['Beg'].values

# (2) Check the Duration of Blinks
print("\n(2) Check The Duration Of Blink Events and Log necessary Steps!")
log_file.write("\n(2) Check The Duration Of Blink Events and Log necessary Steps!\n")
blinkArtifs = []
for i in range(len(blinkPosL)):
    if blinkPosL['Duration'].iloc[i] < 100:
        print(f"{i} Blink is Too Short - Possibly a Processing Artefact ({blinkPosL['Duration'].iloc[i]})")
        log_file.write(f"{i} Blink is Too Short - Possibly a Processing Artefact ({blinkPosL['Duration'].iloc[i]})\n")
        blinkArtifs.append(i)

    if blinkPosL['Duration'].iloc[i] > 600:
        print(f"{i} Blink is Too Long - Possibly Missing Eye Data")
        log_file.write(f"{i} Blink is Too Long - Possibly Missing Eye Data\n")
        blinkArtifs.append(i)

# (3) Check the Blink Rate
print("\n(3) Check the Blink Rate")
log_file.write("\n(3) Check the Blink Rate\n")
print(f"    Blink Rate Raw:      {len(blinkDF) / (len(RawDF) / 500):5.2f} (b/s)")
log_file.write(f"    Blink Rate Raw:      {len(blinkDF) / (len(RawDF) / 500):5.2f} (b/s)\n")
print(f"    Blink Rate Adjusted: {len(blinkPosL) / (len(ScreenData) / 1000):5.2f} (b/s)")
log_file.write(f"    Blink Rate Adjusted: {len(blinkPosL) / (len(ScreenData) / 1000):5.2f} (b/s)\n")
print(f"    Blink Rate Filtered: {len(blinkPosL[(blinkPosL['Duration'] < 600) & (blinkPosL['Duration'] > 100)]) / (len(ScreenData) / 1000):5.2f} (b/s)")
log_file.write(f"    Blink Rate Filtered: {len(blinkPosL[(blinkPosL['Duration'] < 600) & (blinkPosL['Duration'] > 100)]) / (len(ScreenData) / 1000):5.2f} (b/s)\n")

log_file.write("\n" + "-"*100 + "\n")

# Print a message indicating that the blink checks have been written to the log file
print("Blink checks have been written to the log file.")

In [ ]:
log_file.close()


## Analysis Pipeline


+ Limit All Data to Screen:
+ Filter Out Blinks and Surroundings 

In [ ]:
ScreenData.plot(x='TimePoint', y=['LeftPupil', 'RightPupil'], title="Pupil Diameter Over Time")


In [ ]:
ScreenData
blinkDF

In [ ]:
ScreenData = RawDF[((RawDF['LeftX'].values > 0) & (RawDF['LeftX'].values < gazeCoords[2])) &
                   ((RawDF['LeftY'].values > 0) & (RawDF['LeftY'].values < gazeCoords[3]))].copy()


In [ ]:
ScreenData = RawDF[((RawDF['LeftX'].values > 0) & (RawDF['LeftX'].values < gazeCoords[2])) &
                   ((RawDF['LeftY'].values > 0) & (RawDF['LeftY'].values < gazeCoords[3]))].copy()


downsampled = downsamplePupil(ScreenData).copy()
filteredPupil = removeBlinks(blinkDF, downsampled)



plotPupilTimecourse(downsampled, "Downsampled Pupil Data", blinkDF, saccadesDF, fixationDF, chooseViz='001')

In [ ]:
RawDF2 = RawDF

In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

def interpolate_blinks(blinkDF, ScreenData, inter=50):
    """
    Interpolate blinks from data in a specified interval (default 50 ms).

    Parameters:
    blinkDF (DataFrame): DataFrame containing blink events.
    ScreenData (DataFrame): DataFrame containing screen data.
    inter (int): Interval in milliseconds to extend the blink period.

    Returns:
    DataFrame: ScreenData with interpolated blinks.
    """
    # Expand the blink interval by the specified interval before and after
    blinkDF["Beg_extended"] = blinkDF["Beg"] - inter
    blinkDF["End_extended"] = blinkDF["End"] + inter

    # Create a boolean array to mark blink intervals
    is_blink = np.zeros(len(ScreenData), dtype=bool)

    # Iterate over each blink interval and mark the corresponding time points
    for _, row in blinkDF.iterrows():
        is_blink |= (ScreenData["TimePoint"] >= row.Beg_extended) & (ScreenData["TimePoint"] <= row.End_extended)

    # Identify intervals to interpolate and set longer intervals to NaN
    for _, row in blinkDF.iterrows():
        duration = row.End_extended - row.Beg_extended
        if duration <= 500:
            # Interpolate using quadratic interpolation
            mask = (ScreenData["TimePoint"] >= row.Beg_extended) & (ScreenData["TimePoint"] <= row.End_extended)
            for col in ["LeftPupil", "RightPupil"]:
                valid = ScreenData.loc[~mask, col]
                interp_func = interp1d(valid.index, valid.values, kind='quadratic', fill_value="extrapolate")
                ScreenData.loc[mask, col] = interp_func(ScreenData.loc[mask].index)
        else:
            # Set longer intervals to NaN
            mask = (ScreenData["TimePoint"] >= row.Beg_extended) & (ScreenData["TimePoint"] <= row.End_extended)
            ScreenData.loc[mask, ["LeftPupil", "RightPupil"]] = np.nan

    return ScreenData

def remove_missing_data(ScreenData, inter=50):
    """
    Remove missing data (pupil size = 0) from data in a specified interval (default 50 ms).

    Parameters:
    ScreenData (DataFrame): DataFrame containing screen data.
    inter (int): Interval in milliseconds to extend the missing data period.

    Returns:
    DataFrame: ScreenData with missing data removed.
    """
    # Identify missing data intervals
    missing_data = (ScreenData["LeftPupil"] == 0) | (ScreenData["RightPupil"] == 0)
    missing_intervals = ScreenData.loc[missing_data, "TimePoint"]

    # Create a DataFrame for missing intervals
    missingDF = pd.DataFrame({
        "Beg": missing_intervals - inter,
        "End": missing_intervals + inter
    })

    # Interpolate missing data
    ScreenData = interpolate_blinks(missingDF, ScreenData, inter)

    return ScreenData

In [ ]:
blinkDF

In [ ]:
RawDFDown = downsamplePupil(RawDF).copy()


RawDF2 = interpolate_blinks(blinkDF=blinkDF,ScreenData=RawDFDown, inter=50).copy()
ScreenData = RawDF2[((RawDF2['LeftX'].values > 0) & (RawDF2['LeftX'].values < gazeCoords[2])) &
                   ((RawDF2['RightX'].values > 0) & (RawDF2['RightY'].values < gazeCoords[3]))].copy()

plotPupilTimecourse(RawDF2, "Downsampled Pupil Data", blinkDF, saccadesDF, fixationDF, chooseViz='001')

In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

def interpolate_outliers(ScreenData, threshold=3, max_duration=500, boundary=50):
    """
    Interpolate outliers in pupil size data if their duration is less than max_duration.

    Parameters:
    ScreenData (DataFrame): DataFrame containing screen data.
    threshold (int): Threshold for identifying outliers (default 3 for 3-sigma rule).
    max_duration (int): Maximum duration for interpolation (default 500 ms).
    boundary (int): Interval in milliseconds to extend the outlier period.

    Returns:
    DataFrame: ScreenData with interpolated outliers.
    """
    # Create differential time series for the left eye
    diff_left = np.diff(ScreenData['LeftPupil'], prepend=ScreenData['LeftPupil'].iloc[0])

    # Identify outliers using the 3-sigma rule on the differential time series
    mean_diff_left = np.mean(diff_left)
    std_diff_left = np.std(diff_left)

    outliers_left = abs(diff_left - mean_diff_left) > threshold * std_diff_left

    # Visualize the outliers
    plt.figure(figsize=(12, 6))
    plt.plot(ScreenData['TimePoint'], ScreenData['LeftPupil'], label='Left Pupil Size')
    plt.scatter(ScreenData['TimePoint'][outliers_left], ScreenData['LeftPupil'][outliers_left], color='red', label='Outliers')
    plt.xlabel('TimePoint')
    plt.ylabel('Left Pupil Size')
    plt.title('Left Pupil Size with Outliers')
    plt.legend()
    plt.show()

    # Cluster outliers together
    clusters = []
    current_cluster = []
    for i in range(len(outliers_left)):
        if outliers_left[i]:
            current_cluster.append(i)
        else:
            if current_cluster:
                clusters.append(current_cluster)
                current_cluster = []
    if current_cluster:
        clusters.append(current_cluster)

    # Interpolate or set to NaN based on duration
    for cluster in clusters:
        start_idx = max(0, cluster[0] - boundary)
        end_idx = min(len(ScreenData) - 1, cluster[-1] + boundary)
        duration = ScreenData['TimePoint'].iloc[end_idx] - ScreenData['TimePoint'].iloc[start_idx]
        print(f"Cluster from {start_idx} to {end_idx} with duration {duration} ms")

        if duration <= max_duration:
            # Interpolate using cubic splines
            valid = ScreenData.loc[~outliers_left, 'LeftPupil']
            interp_func = interp1d(valid.index, valid.values, kind='cubic', fill_value="extrapolate")
            print(f"Interpolated values {interp_func(ScreenData.index[start_idx:end_idx + 1])} in indexes {start_idx,end_idx + 1}")
            print(f"{ScreenData.index[start_idx:end_idx + 1]} with {valid.index} and {valid.values}")

            ScreenData.loc[ScreenData.index[start_idx:end_idx + 1], 'LeftPupil'] = interp_func(ScreenData.index[start_idx:end_idx + 1])
        else:
            print(f"Duration too much {duration} in position {cluster[-1]}")
            # Set longer intervals to NaN
            ScreenData.loc[ScreenData.index[start_idx:end_idx + 1], 'LeftPupil'] = np.nan

    return ScreenData

# Example usage
RawDFDown = downsamplePupil(RawDF)
RawDF2 = interpolate_blinks(blinkDF=blinkDF, ScreenData=RawDFDown, inter=50)
plotPupilTimecourse(RawDF2, "Downsampled Pupil Data", blinkDF, saccadesDF, fixationDF, chooseViz='001')
ScreenData = RawDF2[((RawDF2['LeftX'].values > 0) & (RawDF2['LeftX'].values < gazeCoords[2])) &
                   ((RawDF2['RightX'].values > 0) & (RawDF2['RightY'].values < gazeCoords[3]))]

RawDF3 = interpolate_outliers(ScreenData=ScreenData, threshold=3, max_duration=500, boundary=5)
plotPupilTimecourse(RawDF3, "Downsampled Pupil Data with Interpolated Outliers", blinkDF, saccadesDF, fixationDF, chooseViz='001')

In [ ]:
RawDFDown = downsamplePupil(RawDF)
RawDF2 = interpolate_blinks(blinkDF=blinkDF, ScreenData=RawDFDown, inter=40)
plotPupilTimecourse(RawDF2, "Downsampled Pupil Data", blinkDF, saccadesDF, fixationDF, chooseViz='001')


RawDF3 = removeOutliers(RawDF2, threshold=3, max_duration=500, boundary=40, verbose=1)
ScreenData = RawDF3[((RawDF3['LeftX'].values > 0) & (RawDF3['LeftX'].values < gazeCoords[2])) &
                   ((RawDF3['LeftY'].values > 0) & (RawDF3['LeftY'].values < gazeCoords[3]))]

plotPupilTimecourse(ScreenData, "Downsampled Pupil Data with Interpolated Outliers", blinkDF, saccadesDF, fixationDF, chooseViz='001')


In [ ]:
import numpy as np
import pandas as pd

# Parameters
boundary = 10

# Example data (replace with your actual data)
# RawDF = pd.read_csv('your_data.csv')  # Load your data here
# RawDFDown = downsamplePupil(RawDF)  # Downsample your data here

# For demonstration, let's assume RawDFDown is already available
ScreenData2 =RawDFDown
# Calculate the center of the screen
center_x = gazeCoords[2] / 2
center_y = gazeCoords[3] / 2

# Calculate the standard deviation of the LeftX and LeftY values, ignoring NaNs
std_x = 0.5* np.nanstd(RawDF2['LeftX'].values)
std_y =0.5* np.nanstd(RawDF2['LeftY'].values)

# Filter the data to include only points within one standard deviation from the center
filtered_data = ScreenData2[
    (np.abs(ScreenData2['LeftX'].values - center_x) < std_x) &
    (np.abs(ScreenData2['LeftY'].values - center_y) < std_y)
]

# Identify points outside the filtered data
outside_center_mask = ~(
    (np.abs(ScreenData2['LeftX'].values - center_x) < std_x) &
    (np.abs(ScreenData2['LeftY'].values - center_y) < std_y)
)

# Create a mask for the points outside the center and their 50 ms boundaries
boundary_mask = np.zeros(len(ScreenData2), dtype=bool)
for idx in np.where(outside_center_mask)[0]:
    start_idx = max(0, idx - boundary)
    end_idx = min(len(ScreenData2) - 1, idx + boundary)
    boundary_mask[start_idx:end_idx + 1] = True

# Combine the masks to exclude points outside the center and their boundaries
final_mask = ~boundary_mask

# Apply the final mask to filter the data
filtered_data_with_boundary = ScreenData2[final_mask]

# Print the filtered data for debugging
print(filtered_data_with_boundary)
plotPupilTimecourse(filtered_data_with_boundary, "Downsampled Pupil Data with Interpolated Outliers", blinkDF, saccadesDF, fixationDF, chooseViz='001')


In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt


# Parameters
threshold = 3
max_duration = 500
boundary = 50

# Example data (replace with your actual data)
# RawDF = pd.read_csv('your_data.csv')  # Load your data here
# RawDFDown = downsamplePupil(RawDF)  # Downsample your data here

# For demonstration, let's assume RawDFDown is already available
ScreenData = RawDF2[((RawDF2['LeftX'].values > 0) & (RawDF2['LeftX'].values < gazeCoords[2])) &
                   ((RawDF2['RightX'].values > 0) & (RawDF2['RightY'].values < gazeCoords[3]))]

# Create differential time series for the left eye
diff_left = np.diff(ScreenData['LeftPupil'], prepend=ScreenData['LeftPupil'].iloc[0])

# Identify outliers using the 3-sigma rule on the differential time series
mean_diff_left = np.mean(diff_left)
std_diff_left = np.std(diff_left)

outliers_left = abs(diff_left - mean_diff_left) > threshold * std_diff_left

# Visualize the outliers
plt.figure(figsize=(12, 6))
plt.plot(ScreenData['TimePoint'], ScreenData['LeftPupil'], label='Left Pupil Size')
plt.scatter(ScreenData['TimePoint'][outliers_left], ScreenData['LeftPupil'][outliers_left], color='red', label='Outliers')
plt.xlabel('TimePoint')
plt.ylabel('Left Pupil Size')
plt.title('Left Pupil Size with Outliers')
plt.legend()
plt.show()

# Cluster outliers together
clusters = []
current_cluster = []
for i in range(len(outliers_left)):
    if outliers_left[i]:
        current_cluster.append(i)
    else:
        if current_cluster:
            clusters.append(current_cluster)
            current_cluster = []
if current_cluster:
    clusters.append(current_cluster)

# Create a mask for the outliers and their boundaries
mask = np.zeros(len(ScreenData), dtype=bool)
for clust in clusters:
    Beg_extended = ScreenData['TimePoint'].iloc[clust[0]] - boundary
    End_extended = ScreenData['TimePoint'].iloc[clust[-1]] + boundary
    mask |= (ScreenData["TimePoint"] >= Beg_extended) & (ScreenData["TimePoint"] <= End_extended)

# Visualize the outliers
plt.figure(figsize=(12, 6))
plt.plot(ScreenData.loc[mask,'TimePoint'], ScreenData.loc[mask,'LeftPupil'], label='Left Pupil Size')
plt.xlabel('TimePoint')
plt.ylabel('Left Pupil Size')
plt.title('Outliers only with extension')
plt.legend()
plt.show()


# Set outlier values to NaNs
ScreenData.loc[mask, 'LeftPupil'] = np.nan

# Interpolate using cubic splines
valid_idx = ScreenData['LeftPupil'].dropna().index  # Indices where data is NOT an artefact
valid_x = valid_idx.to_numpy()
valid_y = ScreenData.loc[valid_idx, 'LeftPupil'].to_numpy()

# Create cubic spline function
cs = CubicSpline(valid_x, valid_y, extrapolate=False) # Extrapolate False to account for missing data at the edges

# Interpolate at artefactual indices
ScreenData.loc[mask, 'LeftPupil'] = cs(ScreenData.index[mask])

# Visualize the interpolated data
plt.figure(figsize=(12, 6))
plt.plot(ScreenData['TimePoint'], ScreenData['LeftPupil'], label='Left Pupil Size')
plt.scatter(ScreenData['TimePoint'][mask], ScreenData['LeftPupil'][mask], color='red', label='Interpolated Points')
plt.xlabel('TimePoint')
plt.ylabel('Left Pupil Size')
plt.title('Left Pupil Size with Interpolated Outliers')
plt.legend()
plt.show()

# Iterate through interpolated clusters and check if any exceed max_duration
for clust in clusters:
    Beg_extended = ScreenData['TimePoint'].iloc[clust[0]] - boundary
    End_extended = ScreenData['TimePoint'].iloc[clust[-1]] + boundary
    mask = (ScreenData["TimePoint"] >= Beg_extended) & (ScreenData["TimePoint"] <= End_extended)
    duration = ScreenData['TimePoint'][mask].iloc[-1] - ScreenData['TimePoint'][mask].iloc[0]
    if duration > max_duration:
        print(f"Cluster from {Beg_extended} to {End_extended} exceeds max_duration with duration {duration} ms")
        ScreenData.loc[mask, 'LeftPupil'] = np.nan

# Visualize the final data
plt.figure(figsize=(12, 6))
plt.plot(ScreenData['TimePoint'], ScreenData['LeftPupil'], label='Left Pupil Size')
plt.xlabel('TimePoint')
plt.ylabel('Left Pupil Size')
plt.title('Left Pupil Size after Removing Long Clusters')
plt.legend()
plt.show()

# Print the mask for debugging
print(mask)




In [ ]:
std_x

In [ ]:

ScreenData.plot(x='TimePoint', y=['LeftPupil', 'RightPupil'], title="Pupil Diameter Over Time")

In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

def interpolate_blinks_and_outliers(ScreenData, blinkDF=None, inter=50, threshold=3, max_duration=500):
    """
    Interpolate blinks and outliers in pupil size data if their duration is less than max_duration.

    Parameters:
    ScreenData (DataFrame): DataFrame containing screen data.
    blinkDF (DataFrame): DataFrame containing blink events (optional).
    inter (int): Interval in milliseconds to extend the blink period.
    threshold (int): Threshold for identifying outliers (default 3 for 3-sigma rule).
    max_duration (int): Maximum duration for interpolation (default 500 ms).

    Returns:
    DataFrame: ScreenData with interpolated blinks and outliers.
    """
    if blinkDF is not None:
        # Expand the blink interval by the specified interval before and after
        blinkDF["Beg_extended"] = blinkDF["Beg"] - inter
        blinkDF["End_extended"] = blinkDF["End"] + inter

        # Create a boolean array to mark blink intervals
        is_blink = np.zeros(len(ScreenData), dtype=bool)

        # Use vectorized operations to mark blink intervals
        for beg, end in zip(blinkDF["Beg_extended"], blinkDF["End_extended"]):
            is_blink |= (ScreenData["TimePoint"] >= beg) & (ScreenData["TimePoint"] <= end)

        # Identify intervals to interpolate and set longer intervals to NaN
        for beg, end in zip(blinkDF["Beg_extended"], blinkDF["End_extended"]):
            duration = end - beg
            mask = (ScreenData["TimePoint"] >= beg) & (ScreenData["TimePoint"] <= end)
            if duration <= max_duration:
                # Interpolate using quadratic interpolation
                for col in ["LeftPupil", "RightPupil"]:
                    valid = ScreenData.loc[~mask, col]
                    interp_func = interp1d(valid.index, valid.values, kind='quadratic', fill_value="extrapolate")
                    ScreenData.loc[mask, col] = interp_func(ScreenData.loc[mask].index)
            else:
                # Set longer intervals to NaN
                ScreenData.loc[mask, ["LeftPupil", "RightPupil"]] = np.nan

    # Identify outliers using the 3-sigma rule
    mean_left = np.mean(ScreenData['LeftPupil'])
    std_left = np.std(ScreenData['LeftPupil'])
    mean_right = np.mean(ScreenData['RightPupil'])
    std_right = np.std(ScreenData['RightPupil'])

    outliers_left = abs(ScreenData['LeftPupil'] - mean_left) > threshold * std_left
    outliers_right = abs(ScreenData['RightPupil'] - mean_right) > threshold * std_right

    # Combine outliers for both eyes
    outliers = outliers_left | outliers_right

    # Cluster outliers together
    clusters = []
    current_cluster = []
    for i in range(len(outliers)):
        if outliers.iloc[i]:
            current_cluster.append(i)
        else:
            if current_cluster:
                clusters.append(current_cluster)
                current_cluster = []
    if current_cluster:
        clusters.append(current_cluster)

    # Interpolate or set to NaN based on duration
    for cluster in clusters:
        duration = ScreenData['TimePoint'].iloc[cluster[-1]] - ScreenData['TimePoint'].iloc[cluster[0]]
        if duration <= max_duration:
            # Interpolate using quadratic interpolation
            for col in ["LeftPupil", "RightPupil"]:
                valid = ScreenData.loc[~outliers, col]
                interp_func = interp1d(valid.index, valid.values, kind='quadratic', fill_value="extrapolate")
                ScreenData.loc[cluster, col] = interp_func(ScreenData.loc[cluster].index)
        else:
            # Set longer intervals to NaN
            ScreenData.loc[cluster, ["LeftPupil", "RightPupil"]] = np.nan

    return ScreenData

# Example usage
ScreenData2 = interpolate_blinks_and_outliers(ScreenData=RawDF, blinkDF=blinkDF, inter=50).copy()

plotPupilTimecourse(ScreenData2, "Downsampled Pupil Data", blinkDF, saccadesDF, fixationDF, chooseViz='001')